<a href="https://colab.research.google.com/github/ka1shi/Toxic_comment_classification_challenge/blob/master/Toxic_comment_testing_v1_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#LSTM

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np 
import pandas as pd 

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D

from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence

from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
 
from keras.models import Model, Sequential
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [0]:
def load_data(filename):
    """
    
    Load Fake news dataset
    
    Input:
    filename : CSV file that contains data
    
    Output:
    data : Fake news data in pandas dataframe
    """
    
    #data = pd.read_csv(filename, sep='\t')
    data = pd.read_csv(filename, encoding = "ISO-8859-1")
    #data = pd.read_csv(filename, sep='\t', encoding = "ISO-8859-1", low_memory=False) 
    
    return data

In [0]:
filename1 = '/content/drive/My Drive/jigsaw-toxic-comment-classification-challenge/train1.csv'
filename2 = '/content/drive/My Drive/jigsaw-toxic-comment-classification-challenge/test1.csv'

train = load_data(filename1)
test = load_data(filename2)

In [0]:
EMBEDDING_FILE = '/content/drive/My Drive/glove.6B/glove.6B.300d.txt'

In [0]:
train.apply(lambda x: sum(x.isnull()), axis=0)

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [0]:
test.apply(lambda x: sum(x.isnull()), axis=0)

id              0
comment_text    0
dtype: int64

In [0]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [0]:
X_train = train["comment_text"].str.lower()
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

X_test = test["comment_text"].str.lower()

In [0]:
max_features=100000
maxlen=150
embed_size=300

In [0]:
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)

In [0]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [0]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:

sequence_input = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(LSTM(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
preds = Dense(6, activation="sigmoid")(x)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


'\n\n\nmodel = Sequential()\nmodel.add(Embedding(max_features, embed_size ,weights = [embedding_matrix], input_length = maxlen, trainable = False))\nmodel.add(SpatialDropout1D(0.2))\nmodel.add(Bidirectional(LSTM(128, dropout = 0.1, recurrent_dropout = 0.1)))\nmodel.add(Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform"))\navg_pool = GlobalAveragePooling1D()\nmax_pool = GlobalMaxPooling1D()\nconcatenated = concatenate([avg_pool, max_pool])\n  \nmodel.add(concatenated)\nmodel.add(Dense(units=6))\nmodel.add(Activation("sigmoid"))\nmodel.compile(loss="binary_crossentropy", optimizer="adam", metrics=[\'accuracy\'])\n    '

In [0]:
print (model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 150, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 150, 256)     439296      spatial_dropout1d_1[0][0]        
____________________________________________________________________________________________

In [0]:
batch_size = 128
epochs = 4
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

In [0]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/4
143613/143613 [==============================] - 2182s 15ms/step - loss: 0.0632 - acc: 0.9784 - val_loss: 0.0517 - val_acc: 0.9811
Epoch 2/4
143613/143613 [==============================] - 2216s 15ms/step - loss: 0.0484 - acc: 0.9822 - val_loss: 0.0472 - val_acc: 0.9821
Epoch 3/4
143613/143613 [==============================] - 2259s 16ms/step - loss: 0.0445 - acc: 0.9833 - val_loss: 0.0457 - val_acc: 0.9832
Epoch 4/4
143613/143613 [==============================] - 2227s 16ms/step - loss: 0.0418 - acc: 0.9841 - val_loss: 0.0459 - val_acc: 0.9831


In [0]:
y_pred = model.predict(x_test,batch_size=1024,verbose=1)

153164/153164 [==============================] - 690s 5ms/step


In [0]:
submission = pd.read_csv('/content/drive/My Drive/jigsaw-toxic-comment-classification-challenge/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission_L.csv', index=False)

In [0]:
model.save_weights('Toxic_comment_testing_v1_weights.h5')

In [0]:
model.save('Toxic_comment_testing_v1.h5')